In [13]:
import feather
import numpy as np 
import pandas as pd
from sklearn.preprocessing import (OneHotEncoder,StandardScaler)
from sklearn.pipeline import Pipeline

from sklearn.model_selection import (
    GridSearchCV)

In [73]:
data_wide_train= feather.read_dataframe('walmart_wide_train.feather')
data_original_train=pd.read_csv("train.csv")

data_wide_test= feather.read_dataframe('walmart_wide_test.feather')
data_original_test=pd.read_csv("test.csv")

In [74]:
data_wide_train

,trip_type,scan_count,department_description_1-HR PHOTO,department_description_ACCESSORIES,department_description_AUTOMOTIVE,department_description_BAKERY,department_description_BATH AND SHOWER,department_description_BEAUTY,department_description_BEDDING,department_description_BOOKS AND MAGAZINES,...,department_description_WIRELESS,regreso,variedad,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1,1,0,0,0,0,0,0
1,23.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2,1,0,0,0,0,0,0
2,19.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,23,1,0,0,0,0,0,0
3,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3,1,0,0,0,0,0,0
4,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,18.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7,0,0,0,1,0,0,0
95670,15.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,1.0,0.0,5,0,0,0,1,0,0,0
95671,32.0,17.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,13,0,0,0,1,0,0,0
95672,32.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,17,0,0,0,1,0,0,0


In [39]:
from scipy import sparse
from scipy.sparse import hstack


In [40]:
def create_sparse_matrix(original_data):
    dummies = pd.get_dummies(original_data.FinelineNumber)
    dummies = dummies[data_original[data_original.FinelineNumber < 200].FinelineNumber.unique()]
    data_visit = original_data[["VisitNumber"]]
    dummy_data = pd.concat([dummies, data_visit], axis=1)
    dummy_data = dummy_data.groupby("VisitNumber")
    total_dummy_data = dummy_data.aggregate(np.sum)

    dummies = pd.get_dummies(original_data.FinelineNumber)
    for x in range(200,9000,100):
        dummies1 = dummies[data_original[(data_original.FinelineNumber >= x) & (data_original.FinelineNumber < x + 100)].FinelineNumber.unique()]
        data_visit = original_data[["VisitNumber"]]
        dummy_data = pd.concat([dummies1, data_visit], axis=1)
        dummy_data = dummy_data.groupby("VisitNumber")
        dummy_data = dummy_data.aggregate(np.sum)
        total_dummy_data = hstack((total_dummy_data, dummy_data))
    return total_dummy_data

In [41]:
#Creo la spare matrix de finite number
spare_matrix_finite_train = create_sparse_matrix(data_original_train)


KeyError: '[45.0] not in index'

In [43]:
spare_matrix_finite_test


NameError: name 'spare_matrix_finite_test' is not defined

In [75]:
#Creo la spare matrix de mis datos wide

data_sparse_train = sparse.bsr_matrix(data_wide_train.astype(np.float))


In [76]:
data_sparse_train

<95674x78 sparse matrix of type '<class 'numpy.float64'>'
	with 702775 stored elements (blocksize = 1x1) in Block Sparse Row format>

In [77]:
#Finalmente hago un merge
data_total_train = hstack((data_sparse_train, sparse_matrix_10000))


In [78]:
data_total_train

<95674x5011 sparse matrix of type '<class 'numpy.float64'>'
	with 1228246 stored elements in COOrdinate format>

In [51]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from numpy import ndarray
import scipy



In [79]:
mytrain, mytest = train_test_split(data_total_train, test_size = .4)


In [80]:

dtrain = xgb.DMatrix(np.asarray(scipy.sparse.bsr_matrix.todense(mytrain[:,1:])), 
                     label = np.asarray(scipy.sparse.bsr_matrix.todense(mytrain[:,0:1])))
dtest = xgb.DMatrix(np.asarray(scipy.sparse.bsr_matrix.todense(mytest[:,1:])), 
                     label = np.asarray(scipy.sparse.bsr_matrix.todense(mytest[:,0:1])))

In [81]:
data_total_train

<95674x5011 sparse matrix of type '<class 'numpy.float64'>'
	with 1228246 stored elements in COOrdinate format>

In [82]:

num_round = 200
param = {'objective': 'multi:softprob', 'num_class':38, 
         'eval_metric': 'mlogloss', "max_delta_step": 5}
param1 = {'objective': 'multi:softprob', 'num_class':38, 
          "max_delta_step": 5}
watchlist = [(dtrain,'train'), (dtest, 'eval')]

In [ ]:

bst = xgb.train(param, dtrain, num_round, watchlist, 
                early_stopping_rounds=3)

[0]	train-mlogloss:2.42726	eval-mlogloss:2.4519
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 3 rounds.
[1]	train-mlogloss:1.83689	eval-mlogloss:1.88237
[2]	train-mlogloss:1.5669	eval-mlogloss:1.62325
[3]	train-mlogloss:1.38537	eval-mlogloss:1.45217
[4]	train-mlogloss:1.25491	eval-mlogloss:1.33122
[5]	train-mlogloss:1.15508	eval-mlogloss:1.24004
[6]	train-mlogloss:1.07821	eval-mlogloss:1.17096
[7]	train-mlogloss:1.01481	eval-mlogloss:1.11465
[8]	train-mlogloss:0.963385	eval-mlogloss:1.0696
[9]	train-mlogloss:0.921218	eval-mlogloss:1.03324
[10]	train-mlogloss:0.885203	eval-mlogloss:1.00281
[11]	train-mlogloss:0.853278	eval-mlogloss:0.9763
[12]	train-mlogloss:0.826087	eval-mlogloss:0.953954
[13]	train-mlogloss:0.80187	eval-mlogloss:0.935002
[14]	train-mlogloss:0.780809	eval-mlogloss:0.919263
[15]	train-mlogloss:0.762155	eval-mlogloss:0.904862
[16]	train-mlogloss:0.744445	eval-mlogloss:0.891557
[

[155]	train-mlogloss:0.339517	eval-mlogloss:0.719279
[156]	train-mlogloss:0.33839	eval-mlogloss:0.719279
[157]	train-mlogloss:0.33738	eval-mlogloss:0.718988
[158]	train-mlogloss:0.336376	eval-mlogloss:0.71894
[159]	train-mlogloss:0.335451	eval-mlogloss:0.718951
[160]	train-mlogloss:0.334284	eval-mlogloss:0.718846
[161]	train-mlogloss:0.333284	eval-mlogloss:0.718767
[162]	train-mlogloss:0.332242	eval-mlogloss:0.71858
[163]	train-mlogloss:0.331171	eval-mlogloss:0.718486
[164]	train-mlogloss:0.330118	eval-mlogloss:0.718429
[165]	train-mlogloss:0.329123	eval-mlogloss:0.718372
[166]	train-mlogloss:0.328124	eval-mlogloss:0.718297
[167]	train-mlogloss:0.326972	eval-mlogloss:0.718144
[168]	train-mlogloss:0.326162	eval-mlogloss:0.717828
[169]	train-mlogloss:0.325275	eval-mlogloss:0.717712
[170]	train-mlogloss:0.324372	eval-mlogloss:0.717827
[171]	train-mlogloss:0.323546	eval-mlogloss:0.717751
[172]	train-mlogloss:0.322612	eval-mlogloss:0.717663
[173]	train-mlogloss:0.321683	eval-mlogloss:0.7175

In [ ]:
my_test = xgb.DMatrix(np.asarray(scipy.sparse.bsr_matrix.todense(mytest)[:,1:]))
test_predictions = bst.predict(my_test)

In [ ]:
log_loss(scipy.sparse.bsr_matrix.todense(mytest[:,0:1]), test_predictions).round(5)
